In [ ]:
# Importing the required library
import numpy as np  # numerical computation
import nltk         # natural language processing
import string       # it can process strings in python
import random
f = open ("delete.txt", "r", errors = "ignore")
raw_doc = f.read()
# converts text to lowercase
raw_doc = raw_doc.lower()
# using the punkt tokenizer -- pre trainned tokenizer to build model around whats required and give an answer
nltk.download("punkt")
# using the WordNet dictionary
nltk.download("wordnet")
# convert doc to list of sentences
sent_tokens = nltk.sent_tokenize(raw_doc)
# convert doc to list of words
word_tokens = nltk.word_tokenize(raw_doc)
sent_tokens[:2]
# two sentences were printed
word_tokens[:2]
# text processing
lemmer = nltk.stem.WordNetLemmatizer()
# wordnet is a semantically oriented dictionary of english included in the nltk
def lem_tokens(tokens):
    return [lemmer.lemmatize(tokens) for token in tokens]
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)
def lem_normalize(text):
    return lem_tokens (nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# defining the greeting function
greet_inputs = ("hello", "hi", "greetings", "sup", "what's up", "hey", "good morning", "good evening", "good afternoon")
greet_response = ["hi", "greetings", "hey", "long time no see", "hi there!", "*nods*", "I am glad! you are talking to me"]
def greet(sentence):
    for words in sentence.split():
        if words.lower() is greet_inputs:
            return random.choice(greet_response)

# response generation
from sklearn.feature_extraction.text import TfidfVectorizer   # (tf)term frequency(how many times each word is repeated in the corpus) and inword document frequency(idf)(how rare is the occurance of word in the corpus) 
from sklearn.metrics.pairwise import cosine_similarity         # normalized vectors for the bag of words
def response(user_response):
    robo1_response = ''
    TfidfVec = TfidfVectorizer(tokenizer = lem_normalize, stop_words = "english")
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf == 0):             # so that the bot knows that if the given data or word is not present in the corpus what he is supposed to do with it
        robo1_response = robo1_response + "I am sorrt! i don't understand you"
        return robo1_response
    else:
        robo1_response = robo1_response + sent_tokens[idx]
        return robo1_response
    
# defining conversation start/end protocols
flag = True 
print ("BOT: My name is Stark. Let's have a conversation! Also, if you want to exit any time, just type 'Bye'!")
while (flag == True):
    user_response = input()
    user_response = user_response.lower()
    if(user_response != 'bye'):
        if (user_response == 'thanks' or user_response == 'thank you'):
            flag = False
            print("BOT: You are welcome")
        else:
            if(greet(user_response)!= None):
                print("BOT:" + greet(user_response))
            else:
                sent_tokens.append(user_response)
                word_tokens = word_tokens + nltk.word_tokenize(user_response)
                final_words = list(set(word_tokens))
                print("BOT: ", end = "")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag = False
        print("BOT: Goodbye! take care <3")

